# Język Python - Wykład 7.

## Jeszcze więcej o testowaniu naszej aplikacji, modułu, package'u

In [ ]:
import timeit

In [ ]:
print timeit.timeit('for i in range(100000): i', number=100)

In [ ]:
print timeit.timeit('char in text', setup='text = "sample string"; char = "g"')
print timeit.timeit('text.find(char)', setup='text = "sample string"; char = "g"')
print timeit.timeit('len([a==char for a in text])>0', setup='text = "sample string"; char = "g"')


In [ ]:
# Jak szybkie sa metody laczenia stringow?
codes = {
    '" ".join(chars)',
    'reduce(lambda a, b : a + " " + b, chars)',
    'for a in chars: word += a + " ";'
}

for code in codes:
    print code
    print timeit.timeit(code, setup='chars = ["a"] * 1000; word = ""', number=1000)
    print

In [ ]:
for length in [100, 1000, 10000, 100000]:

    codes = [
        '"-".join(str(n) for n in xrange('+str(length)+'))',
        '"-".join([str(n) for n in xrange('+str(length)+')])',
        '"-".join(map(str, xrange('+str(length)+')))'
    ]

    scores = {}
    
    for code in codes:
        print code
        score = timeit.timeit(code, number=1000000 / length)
        scores[code] = score
        print score
        print
    
    best = None
    for code in scores:
        if (best is None) or (scores[code] < scores[best]):
            best = code
            
    print
    print 'For ', length, 'numbers ', best, 'is the fastest'
    print
    print

#### code profiling

In [ ]:
import cProfile

import time

def sleep(n):
    time.sleep(n)

def bar():
    time.sleep(0.1)
    return 1

def foo():
    x = 0
    for i in range(100):
        x += bar()
    sleep(3)
    return x

cProfile.run('foo()')

In [ ]:
import cProfile
def gen(n):
    return "-".join(str(n) for n in xrange(n))

def lst(n):
    return "-".join([str(n) for n in xrange(n)])

print cProfile.run('gen(1000000)')
print cProfile.run('lst(1000000)')

In [ ]:
import cProfile

def do_cprofile(func):
    def profiled_func(*args, **kwargs):
        profile = cProfile.Profile()
        try:
            profile.enable()
            result = func(*args, **kwargs)
            profile.disable()
            return result
        finally:
            profile.print_stats()
    return profiled_func

import time

@do_cprofile
def sleep(n):
    time.sleep(n)

#@do_cprofile
def bar():
    time.sleep(0.1)
    return 1

def foo():
    x = 0
    for i in range(100):
        x += bar()
    sleep(3)
    return x

foo()

#### UWAGA - wymaga zewnętrznej biblioteki "line_profiler"

    pip install line_profiler

In [ ]:
from line_profiler import LineProfiler

def do_profile(follow=[]):
    def inner(func):
        def profiled_func(*args, **kwargs):
            try:
                profiler = LineProfiler()
                profiler.add_function(func)
                for f in follow:
                    profiler.add_function(f)
                profiler.enable_by_count()
                return func(*args, **kwargs)
            finally:
                profiler.print_stats()
        return profiled_func
    return inner
    
def sleep(n):
    for i in range(n):
        x = 17 ** 4
        time.sleep(1)

def bar():
    time.sleep(0.1)
    return 1

@do_profile(follow=[sleep, bar])
def foo():
    x = 0
    for i in range(100):
        x += bar()
    sleep(3)
    return x

foo()

In [ ]:
@do_profile()
def primes(n): 
    if n==2:
        return [2]
    elif n<2:
        return []
    s=range(3,n+1,2)
    mroot = n ** 0.5
    half=(n+1)/2-1
    i=0
    m=3
    while m <= mroot:
        if s[i]:
            j=(m*m-3)/2
            s[j]=0
            while j<half:
                s[j]=0
                j+=m
        i=i+1
        m=2*i+3
    return [2]+[x for x in s if x]

null = primes(1000)

## Tworzenie własnych modułów

In [ ]:
import math

In [ ]:
from math import sqrt

In [ ]:
from numpy.linalg import lstsq

Też chce taki!

In [ ]:
import my_module

In [ ]:
dir(my_module)

In [ ]:
with open('my_module/__init__.py') as f:
    print f.read()

In [ ]:
from my_module import cars

In [ ]:
dir(cars)

In [ ]:
with open('my_module/cars.py') as f:
    print f.read()

In [ ]:
fiat = cars.Fiat()

In [ ]:
from my_module.cars import Fiat
fiat = Fiat()

In [ ]:
from my_module.utils import io # Konstruktory modulow

In [ ]:
with open('my_module/utils/__init__.py') as f:
    print f.read()

A co jeśli nie ma `__init__.py`?

In [ ]:
from my_module import test

Importowanie względne...

In [ ]:
from my_module.utils import visualization

In [ ]:
dir(visualization)

In [ ]:
with open('my_module/utils/visualization.py') as f:
    print f.read()

Czasami trzeba przeładować moduł

In [ ]:
import my_module
mod1 = reload(my_module)
mod2 = reload(my_module)

print mod1
print dir(mod2)

Mamy moduł `optimizers`, gdzie trzymamy różne metody minimalizacji funkcji

    optimizers/
        __init__.py
        linear/
            __init__.py
            lsqr.py
        meta/
            __init__.py
            genetic.py
            ants.py
            
Jednak użytkownik nie powinien uczyć się na pamięć naszej skomplikowanej struktury, powinien raczej używać:

    from optimizers import GeneticAlgorithm # zamiast from optimizers.meta.genetic import GeneticAlgorithm
    
W tym celu w `__init__` czasami "wystawia" się klasy, np. poprzez

    from linear.lsqr import LeastSquares
    from meta.genetic import GeneticAlgorithm
    from meta.ants import AntColony
    
W pliku `__init__` nie powinniśmy robić **niczego** poza inicjalizacją / wystawieniem klas/funkcji

In [ ]:
import numpy
help(numpy)

Jak zdefiniować własny help?

In [ ]:
help(my_module)

In [ ]:
from my_module import cars
help(cars)

Co trafia do `help`?

* Docstring z początku pliku - pierwsza linijka to opis krótki, to co później to DESCRIPTION
* Wszystkie zmienne (DATA), funkcje (FUNCTIONS), klasy (CLASSES), podmoduły (PACKAGE CONTENTS)
* Docstringi funkcji/klas trafiają w odpowiednie pola
* Niektóre "magiczne" zmienne, tzn. \_\_author\_\_, \_\_version\_\_, \_\_credits\_\_, ... są dodatkowo wyświetlane jako osobne pola

In [ ]:
print 'NAME', cars.__name__
print 'DOC', cars.__doc__
print 'DIR', dir(cars)
print
print 'FIAT DOC', cars.Fiat.__doc__

#### Generowanie dokumentacji "offline"

<a href="my_module.html">docs</a>

Dla bardziej zaawansowanych: http://sphinx-doc.org/